In [156]:
import random

import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.nn import functional as F

MAX_LENGTH = 1024
INPUT_SOS = '<|context|>'
INPUT_EOS = '<|endofcontext|>'
OUTPUT_SOS = '<|belief|>'
OUTPUT_EOS = '<|endofbelief|>'

In [132]:
with open('../resources/gpt2/train.history_belief') as fp:
    raw_train_text = fp.read()

In [133]:
raw_train_text = raw_train_text.replace('<|endofcontext|>', ' <|endofcontext|>')
raw_train_text = raw_train_text.replace('<|endoftext|>', '')

In [134]:
train_text = raw_train_text.split('\n')

In [135]:
input_text = []
beliefs = []

for text in train_text:
    split_index = text.find('<|belief|>')  
    input_text.append(text[:split_index-1].split())
    beliefs.append(text[split_index:].split())

In [136]:
train_data = pd.DataFrame(data={'input_text': input_text, 'beliefs': beliefs})

In [60]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.n_words = 0
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [128]:
input_lang = Lang('input_lang')
output_lang = Lang('output_lang')
for word in raw_train_text.split():
    input_lang.addWord(word)
    output_lang.addWord(word)
    
input_lang.EOS = INPUT_EOS
output_lang.EOS = OUTPUT_EOS

In [63]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [64]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [67]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [146]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(lang.word2index[lang.EOS])
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [152]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[output_lang.word2index[OUTPUT_SOS]]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == output_lang.word2index[OUTPUT_EOS]:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [73]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [143]:
train_tuples = [t[1:] for t in train_data.itertuples()]

In [148]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_tuples))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [84]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [151]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[output_lang.word2index[OUTPUT_SOS]]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == output_lang.word2index[OUTPUT_EOS]:
                decoded_words.append(output_lang.word2index[OUTPUT_EOS])
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [137]:
with open('../resources/gpt2/val.history_belief') as fp:
    raw_val_text = fp.read()

In [138]:
raw_val_text = raw_val_text.replace('<|endofcontext|>', ' <|endofcontext|>')
raw_val_text = raw_val_text.replace('<|endoftext|>', '')

In [139]:
val_text = raw_val_text.split('\n')

In [140]:
input_text = []
beliefs = []

for text in val_text:
    split_index = text.find('<|belief|>')  
    input_text.append(text[:split_index-1].split())
    beliefs.append(text[split_index:].split())

In [141]:
val_data = pd.DataFrame(data={'input_text': input_text, 'beliefs': beliefs})

In [142]:
val_tuples = [t[1:] for t in val_data.itertuples()]

In [119]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(val_tuples)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [120]:
device = None

In [160]:
hidden_size = 4
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 100, print_every=10)

0m 0s (- 0m 7s) (10 10%) 9.7230
0m 1s (- 0m 7s) (20 20%) 9.3331
0m 2s (- 0m 6s) (30 30%) 8.6520
0m 3s (- 0m 5s) (40 40%) 7.9740
0m 4s (- 0m 4s) (50 50%) 7.1566
0m 5s (- 0m 3s) (60 60%) 6.6511
0m 6s (- 0m 2s) (70 70%) 5.8594
0m 7s (- 0m 1s) (80 80%) 5.7120
0m 8s (- 0m 0s) (90 90%) 5.8246
0m 10s (- 0m 0s) (100 100%) 5.2826


In [161]:
evaluateRandomly(encoder1, attn_decoder1)

> ['<|context|>', '<|user|>', 'good', 'morning', ',', 'i', 'am', 'looking', 'for', 'some', 'information', 'in', 'cambridge', '.', 'can', 'you', 'help', 'me', '?', '<|system|>', 'sure', 'thing', '.', 'what', 'type', 'of', 'information', 'are', 'you', 'looking', 'for', '?', '<|user|>', 'can', 'you', 'tell', 'me', 'about', 'the', 'museums', 'in', 'the', 'center', 'of', 'town', '?', '<|system|>', 'there', 'are', '11', 'in', 'the', 'centre', 'of', 'town', ',', 'how', 'about', 'the', 'castle', 'galleries', '?', '<|user|>', 'what', 's', 'the', 'entrance', 'fee', 'for', 'castle', 'galleries', '?', '<|system|>', 'it', 'is', 'free', 'to', 'enter', '!', 'enjoy', 'yourself', '!', '<|user|>', 'could', 'i', 'get', 'the', 'phone', 'number', 'and', 'postcode', 'as', 'well', '?', '<|system|>', 'of', 'course', '.', 'the', 'postal', 'code', 'is', 'cb23bj', ',', 'and', 'the', 'phone', 'number', 'is', '01223307402', '.', 'anything', 'else', 'you', 'need', 'today', '?', '<|user|>', 'yes', ',', 'i', 'would',

KeyError: 'caqb4i7n'